Try to build a neural network to predict price of a laptop. Will use a simple linear regression model and compare accuracy later 

In [13]:
import pandas as pd 
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


Splitting the data 

In [14]:
X_train = pd.read_csv('X_train_pre.csv')
y_train = pd.read_csv('y_train_pre.csv')

X_test = pd.read_csv('X_test_pre.csv')
y_test = pd.read_csv('y_test_pre.csv')



In [15]:
model1 = LinearRegression()

def model2():
    return torch.nn.Sequential(
    torch.nn.Linear(X_train.shape[1] , 30),
    torch.nn.ReLU(),
    torch.nn.Linear(30,10),
    torch.nn.ReLU(),
    torch.nn.Linear(10,1)
)
def model3():
    return  torch.nn.Sequential(
    torch.nn.Linear(X_train.shape[1] , 20),
    torch.nn.ReLU(),
    torch.nn.Linear(20,1)
)
def model4() :
    return  torch.nn.Sequential(
    torch.nn.Linear(X_train.shape[1] , 70),
    torch.nn.ReLU(),
    torch.nn.Linear(70,30),
    torch.nn.ReLU(),
    torch.nn.Linear(30,5),
    torch.nn.ReLU(),
    torch.nn.Linear(5,1)
)

model_class = [ model2, model3 , model4]
model_names = ["2 layers", "1 layer", '3 layers']

Function to train model  

In [16]:
def train_one(model, train_dl, val_dl, max_epochs=500, patience=20):
    opt = torch.optim.Adam(model.parameters())
    loss_fn =torch.nn.MSELoss()
    best_val = float("inf")
    best_state = None
    no_improve = 0

    for epoch in range(max_epochs):
        model.train()
        for xb, yb in train_dl: # training mode 
            opt.zero_grad() #zero gradient
            pred = model(xb).squeeze(1) #Predicton
            loss = loss_fn(pred, yb) #COmpute the loss
            loss.backward() #compute the gradients
            opt.step()   # update the weights

        model.eval() 
        val_loses=[]
        with torch.no_grad(): # no gradient tracking
            for xb , yb in val_dl:
                pred = model(xb).squeeze(1) #predict with current weights
                val_loses.append(loss_fn(pred, yb).item()) #add loss to prediction list
        val_mse = np.mean(val_loses) # find mean of the list
        if val_mse < best_val - 1e-6: 
            best_val = val_mse
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                break
    if best_state is not None:
        model.load_state_dict(best_state)
    return np.sqrt(best_val)  # best RMSE

print(X_train.index)
print(y_train.index)




RangeIndex(start=0, stop=1082, step=1)
RangeIndex(start=0, stop=1082, step=1)


In [17]:
# Convert DataFrames to numpy arrays before creating tensors
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
Y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).squeeze(1)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
Y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).squeeze(1)

# Create a Dataset and DataLoader
full_dataset = torch.utils.data.TensorDataset(X_train_tensor, Y_train_tensor)
full_loader = torch.utils.data.DataLoader(full_dataset, batch_size=32, shuffle=True)

# Training loop
def train_full_model(model, loader, epochs=500, lr=1e-3):
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = torch.nn.MSELoss()
    for epoch in range(epochs):
        model.train()
        for xb, yb in loader:
            opt.zero_grad()
            pred = model(xb).squeeze(1)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()
    return model

# Train the model
neural_model = model4()
neural_model = train_full_model(neural_model, full_loader, epochs=500, lr=1e-3)

neural_model.eval()
with torch.no_grad():
    p=neural_model(X_test_tensor).squeeze(1)
    mse = torch.mean((p - Y_test_tensor)**2)
    se = np.sqrt(mse.item())
print("3 layered model RMSE : ", se)

3 layered model RMSE :  19532.493542812193


In [18]:
model1.fit(X_train,y_train)
preds = model1.predict(X_test)
rms = np.sqrt(np.mean((preds-y_test)**2))
print("Linear regression model pediction : ", rms)


Linear regression model pediction :  19556.846760237382


In [20]:
# Calculate Mean Absolute Percentage Error (MAPE) for 3-layer model predictions
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.array(y_true).ravel()
    y_pred = np.array(y_pred).ravel()
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# Get predictions from the 3-layer neural model
neural_model.eval()
with torch.no_grad():
    preds_3layer = neural_model(X_test_tensor).squeeze(1).cpu().numpy()

mape_3layer = mean_absolute_percentage_error(y_test.values, preds_3layer)
print(f"3-layer model MAPE (%): {mape_3layer:.2f}")

3-layer model MAPE (%): 21.06
